# V11 (gated 82+83): ModShift

Re-runs ModShift on the gated (82+83) dataset to test signal uniqueness after removing depth-inconsistent sectors.


In [ ]:
from pathlib import Path
import json
import sys

tutorial_dir = Path('docs/tutorials/tutorial_toi-5807-incremental').resolve()
sys.path.insert(0, str(tutorial_dir))

import toi5807_shared as sh

sectors_used = [82, 83]
ds = sh.load_dataset()
lc = sh.stitch_pdcsap_sectors(ds, sectors_used)
depth_ppm, depth_err_ppm = sh.estimate_depth_ppm(lc)
candidate = sh.make_candidate(depth_ppm)

session = sh.make_session(stitched=lc, candidate=candidate, network=False, preset='extended')
r = session.run('V11')

print(json.dumps({
  'sectors_used': sectors_used,
  'depth_ppm': depth_ppm,
  'depth_err_ppm': depth_err_ppm,
  'status': r.status,
  'flags': list(r.flags),
  'metrics': r.metrics,
}, indent=2, sort_keys=True))


<details>
<summary><b>Expected Output</b></summary>

```text
{
  "depth_err_ppm": 11.153641839855947,
  "depth_ppm": 264.57118442002604,
  "flags": [],
  "metrics": {
    "baseline_days": 50.98,
    "false_alarm_threshold": 2.516974,
    "fred": 52.981048,
    "n_points": 35459,
    "n_transits_expected": 3,
    "positive_primary_ratio": 0.465421,
    "positive_signal": 1576.18745,
    "primary_signal": 3386.58648,
    "secondary_primary_ratio": 0.332581,
    "secondary_signal": 1126.314328,
    "tertiary_primary_ratio": 0.328932,
    "tertiary_signal": 1113.957353
  },
  "sectors_used": [
    82,
    83
  ],
  "status": "ok"
}
```

</details>


In [ ]:
import json
import matplotlib.pyplot as plt

from tess_vetter.plotting import plot_modshift

step_id = '29_gated_82_83_v11_modshift'
fname = 'V11_modshift.png'

run_out_dir, docs_out_dir = sh.artifact_dirs(step_id=step_id)
run_path = run_out_dir / fname
docs_path = (docs_out_dir / fname) if docs_out_dir is not None else None

out = {'status': r.status, 'flags': list(r.flags)}

if r.status == 'ok':
    fig, ax = plt.subplots(figsize=(9, 5))
    plot_modshift(r, ax=ax)
    ax.set_title('V11 (gated 82+83): ModShift')
    fig.tight_layout()
    fig.savefig(run_path, dpi=160, bbox_inches='tight')
    if docs_path is not None:
        fig.savefig(docs_path, dpi=160, bbox_inches='tight')
    plt.show()
    out['run_plot_path'] = str(run_path)
    out['docs_plot_path'] = str(docs_path) if docs_path is not None else None

print(json.dumps(out, indent=2, sort_keys=True))


<details>
<summary><b>Expected Output (plot cell)</b></summary>

```text
{
  "docs_plot_path": "docs/tutorials/artifacts/tutorial_toi-5807-incremental/29_gated_82_83_v11_modshift/V11_modshift.png",
  "run_plot_path": "persistent_cache/tutorial_toi-5807-incremental/29_gated_82_83_v11_modshift/V11_modshift.png",
  "status": "ok"
}
```

</details>


## Plot

<img src="../artifacts/tutorial_toi-5807-incremental/29_gated_82_83_v11_modshift/V11_modshift.png" width="820" />


<details>
<summary><b>Analysis</b></summary>

- **Flags:** none.
- **Result:** strong primary signal remains, with non-primary peaks (secondary/primary ratio ≈0.33).
- **Why this is useful:** highlights whether there are comparably strong transit-like events at other phases (EB/systematics risk).
- **Interpretation:** non-primary structure persists under gating; treat as caution and corroborate with SWEET and robustness (V16–V19).
- **Next step:** V12 (gated) SWEET.

</details>
